In [4]:
from pyspark.sql import SparkSession

In [9]:
df = spark.read.csv('/Users/hamsinisankaran/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/titanic.csv', header=True,inferSchema=True)

In [10]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [11]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [12]:
my_cols = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare','Embarked'])

In [13]:
#deal with the missing data
my_final_data = my_cols.na.drop()

In [14]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [17]:
#string indexer to convert the string into a num based on categories
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')

#perform one hot encoding 
#A B C 
#[1 0 0]
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [18]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [19]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [20]:
from pyspark.ml.classification import LogisticRegression

In [23]:
from pyspark.ml import Pipeline 

In [24]:
#pipeline - set up the stages to do the coding 
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [37]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,
                            embark_encoder,assembler,log_reg_titanic])

In [38]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

In [39]:
fit_model = pipeline.fit(train_data)

In [40]:
results = fit_model.transform(test_data)

In [41]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [43]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [44]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [47]:
AUC = my_eval.evaluate(results)

In [48]:
AUC

0.771505376344086